In [178]:
from sort import *

In [180]:
import ultralytics
from ultralytics import YOLO
import math

In [183]:
import cv2
import cvzone
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import time
path_of_folder = 'C:/Users/HP/Desktop/jupyterNotebook/projects/photos_of_overspeedCars'
os.makedirs(path_of_folder, exist_ok=True)

In [185]:
import numpy as np
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,BatchNormalization,Dropout
from keras import Sequential
from twilio.rest import Client
import tensorflow_datasets as tfds

In [220]:
speed_limit=2
speeds=0
start_time=0
end_time=0
scale_factor = 0.3
cap = cv2.VideoCapture('C:/Users/HP/Downloads/2103099-uhd_3840_2160_30fps.mp4')
model=YOLO('../YOLO-Weight/yolov8n.pt')
#numbers_of_vehicals=0

classNames = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", 
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", 
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", 
    "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", 
    "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", 
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "potted plant", 
    "bed", "dining table", "toilet", "tv monitor", "laptop", "mouse", "remote", "keyboard", "cell phone", 
    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", 
    "teddy bear", "hair dryer", "toothbrush"
]

In [222]:
mask=cv2.imread('C:/Users/HP/Downloads/r.png')

tracker=Sort(max_age=20,min_hits=3,iou_threshold=0.3)



totalCount=[]



frame_width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("frame_width",frame_width)
print("frame_height",frame_height)

mask=cv2.resize(mask,(frame_width,frame_height))


frame_width 3840
frame_height 2160


In [224]:
# Initialize time tracking dictionary
vehicle_times = {}

while True:
    success, img = cap.read()
    if not success:
        print("End of video or failed to read frame.")
        break

    # Resize image
    new_width = int(frame_width * scale_factor)
    new_height = int(frame_height * scale_factor)
    img_resized = cv2.resize(img, (new_width, new_height))
    mask = cv2.resize(mask, (new_width, new_height))

    # Apply mask
    imgRegion = cv2.bitwise_and(img_resized, mask)

    # YOLO detection
    results = model(imgRegion, stream=True)
    detections = np.empty((0, 5))

    # Draw lines for vehicle tracking
    line_y1 = int(new_height / 2 - 40)
    line_y2 = int(new_height / 2 + 35)
    cv2.line(img_resized, (0, line_y1), (new_width, line_y1), (200, 100, 255), 5)
    cv2.line(img_resized, (0, line_y2), (new_width, line_y2), (0, 0, 255), 5)

    # Process YOLO results
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])

            # Filter relevant classes and confidence
            if conf > 0.3 and cls in [2, 3, 5, 7]:  # Cars, trucks, buses, etc.
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

                # Draw bounding box and label
                cvzone.cornerRect(img_resized, (x1, y1, x2 - x1, y2 - y1), l=9, rt=4, colorR=(255, 0, 255))
                cvzone.putTextRect(img_resized, f'{conf} {cls}', (max(0, x1), max(35, y1)), scale=0.7, thickness=1)

    # Update tracker
    resultsTracker = tracker.update(detections)
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cx, cy = x1 + (x2 - x1) // 2, y1 + (y2 - y1) // 2

        # Draw tracker info
        cvzone.cornerRect(img_resized, (x1, y1, x2 - x1, y2 - y1), l=9, rt=4, colorR=(0, 255, 0))
        cvzone.putTextRect(img_resized, f'ID: {int(id)}', (max(0, x1), max(35, y1)), scale=0.7, thickness=1)
        cv2.circle(img_resized, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        # Track vehicle crossing the first line
        if line_y1 - 20 < cy < line_y1 + 20:
            if id not in vehicle_times:
                vehicle_times[id] = {'start_time': time.time(), 'end_time': None}
            cv2.line(img_resized, (0, line_y1), (new_width, line_y1), (0, 255, 0), 5)

        # Track vehicle crossing the second line
        if line_y2 - 20 < cy < line_y2 + 20 and vehicle_times.get(id, {}).get('start_time') is not None:
            vehicle_times[id]['end_time'] = time.time()

            # Calculate speed if not already calculated
            start_time = vehicle_times[id]['start_time']
            end_time = vehicle_times[id]['end_time']
            if start_time and end_time:
                elapsed_time = end_time - start_time
                distance = 10  # Example: 10 meters between the lines
                speed = distance / elapsed_time * 3.6  # Speed in km/h (convert m/s to km/h)

                print(f"Vehicle ID {id} Speed: {speed:.2f} km/h")
                if speed > speed_limit:
                    photo_name = f"overspeed_{id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
                    cv2.imwrite(os.path.join(path_of_folder, photo_name), img_resized)
                    
                    print(f"Vehicle ID {id} exceeded speed limit! Speed: {speed:.2f} km/h")
                else:
                    print(f"Vehicle ID {id} is within speed limit.")

    # Display count
    cvzone.putTextRect(img_resized, f'Count: {len(vehicle_times)}', (50, 50), scale=1, thickness=2)

    # Show the output
    cv2.imshow("Video", img_resized)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 12 cars, 52.0ms
Speed: 2.0ms preprocess, 52.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 57.2ms
Speed: 8.1ms preprocess, 57.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 53.3ms
Speed: 15.8ms preprocess, 53.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 62.3ms
Speed: 1.5ms preprocess, 62.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 70.6ms
Speed: 14.8ms preprocess, 70.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 67.4ms
Speed: 0.0ms preprocess, 67.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 52.7ms
Speed: 3.2ms preprocess, 52.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 51.8ms
Speed: 3.3ms preprocess, 51.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640

In [ ]:
account_sid = 'ACe8722b25a1c1b7a58bfdc3e1124d43c9'
auth_token = '42bb0aec5fd958190de710444e1a9696'
client = Client(account_sid, auth_token)

# Function to map the predicted class index to a character
def class_to_char(predicted_class):
    if predicted_class < 10:
        return chr(predicted_class + 48)  # Digits 0-9
    elif predicted_class < 36:
        return chr(predicted_class - 10 + 65)  # Uppercase A-Z
    else:
        return chr(predicted_class - 36 + 97)  # Lowercase a-z

# Preprocess the dataset (normalize images)
def preprocess(example):
    image = tf.cast(example['image'], tf.float32) / 255.0
    label = example['label']
    return image, label

# Load and preprocess the EMNIST dataset
emnist_builder = tfds.builder('emnist/byclass')
emnist_builder.download_and_prepare()
emnist = emnist_builder.as_dataset(split='train')
train_dataset = emnist.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(62, activation='softmax')  # 62 classes for digits and letters
])

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, epochs=50)


In [ ]:
def processing_image(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
    img_resized = cv2.resize(img_gray, (28, 28))
    img_array = img_resized / 255.0
    return img_array.reshape(1, 28, 28, 1)

# Function to segment characters from the image
def segment_characters(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])
    char_bboxes = [cv2.boundingRect(c) for c in contours if cv2.contourArea(c) > 100]
    return char_bboxes, binary

# Folder path containing the images

images = os.listdir(path_of_folder)

In [ ]:
for img_name in images:
    img_path = os.path.join(path_of_folder, img_name)
    
    photo = None
    while photo is None:
        photo = cv2.imread(img_path)
        if photo is None:
            print(f"Retrying: Unable to load image {img_path}")

    if photo is None:
        print(f"Error: Unable to load image {img_path}")
        continue

    # Segment characters
    char_bboxes, binary = segment_characters(photo)
    filtered_bboxes = [bbox for bbox in char_bboxes if bbox[2] > 5 and bbox[3] > 5]
    l = []

    # Predict characters
    for bbox in filtered_bboxes:
        x, y, w, h = bbox
        char_img = binary[y:y + h, x:x + w]
        input_img = processing_image(char_img)
        prediction = model.predict(input_img)
        predicted_class = np.argmax(prediction)
        predicted_char = class_to_char(predicted_class)
        l.append(predicted_char)

        # Draw bounding box and text
        cv2.rectangle(photo, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cvzone.putTextRect(photo, predicted_char, (x, y + 30), scale=1, thickness=1, colorR=(0, 0, 255))

    # Combine characters into a plate number
    plate_number = ''.join(l)
    print(f"Detected Plate Number: {plate_number}")

    # Send message
    try:
        message = client.messages.create(
            body=f"The car with number plate {plate_number} has over-speeded.",
            from_='+12314488432',
            to='+919359645981'
        )
        print(f"Message sent with SID: {message.sid}")
    except Exception as e:
        print(f"Failed to send message: {e}")

    # Display the processed image
    cv2.imshow("Segmented Characters", photo)
    cv2.waitKey(0)

cv2.destroyAllWindows()